In [ ]:
!pip install -q sagemaker --upgrade 

In [ ]:
import sagemaker.huggingface

In [ ]:
import sagemaker

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

In [ ]:
import boto3
import os


prefix = 'DEMO-huggingface-finetune-dialog-summarization'
s3_input_train_validation = 's3://{}/{}/train'.format(sagemaker_session_bucket, prefix)

s3_input_train_validation = sagemaker.s3.S3Uploader.upload(local_path='./dataset', desired_s3_uri=s3_input_train_validation)

In [ ]:
s3_input_train_validation

In [ ]:
!pygmentize ./scripts/train_flan_t5_sm_compatible.py 

In [ ]:
from sagemaker.huggingface import HuggingFace
import time

# hyperparameters, which are passed into the training job
hyperparameters={'epochs': 3,                          # number of training epochs
                 'train_batch_size': 8,               # batch size for training
                 'learning_rate': 0.00005,                # learning rate used during training
                }

In [ ]:
# define Training Job Name 
job_name = f'huggingface-finetune-dialog{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'

# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'train_flan_t5_sm_compatible.py', # fine-tuning script used in training jon
    source_dir           = './scripts',       # directory where fine-tuning script is stored
    instance_type        = 'ml.p3.2xlarge',   # instances type used for the training job
    #instance_type        = 'local_gpu',   # instances type used for the training job    
    instance_count       = 1,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    transformers_version = '4.26',           # the transformers version used in the training job
    pytorch_version      = '1.13',           # the pytorch_version version used in the training job
    py_version           = 'py39',            # the python version used in the training job
    hyperparameters      = hyperparameters,   # the hyperparameter used for running the training job
)

In [ ]:
# define a data input dictonary with our uploaded s3 uris

data = {
    'train': s3_input_train_validation,
}

# starting the train job with our uploaded datasets as input
huggingface_estimator.fit(data, wait=False)